In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import cv2
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.preprocessing.image import img_to_array

best_score=0
filename=None
image =None
kp1,kp2, mp= None, None, None
for file in [file for file in os.listdir("/content/drive/MyDrive/SOCOFing/SOCOFing/Real")[:1]]:
        print(file)
        filepath ="/content/drive/MyDrive/SOCOFing/SOCOFing/Real/100__M_Left_index_finger.BMP"
        filepath_sample ="/content/drive/MyDrive/SOCOFing/SOCOFing/Altered/Altered-Easy/"+file.split(".")[0]+'_CR.BMP'
        sample_1=cv2.imread(filepath)
        sample=cv2.imread(filepath_sample)
        print(filepath)
        sift = cv2.SIFT_create()
        keypoint1, descriptor1 = sift.detectAndCompute(sample_1, None)
        keypoint2, descriptor2 = sift.detectAndCompute(sample, None)

        # FLANN parameters
        flann_params = {
            'algorithm': 1,  # FLANN_INDEX_KDTREE
            'trees': 10
        }

        # Create FLANN matcher
        flann = cv2.FlannBasedMatcher(flann_params, {})
        # Perform KNN matching
        matches = flann.knnMatch(descriptor1, descriptor2, k=2)

                # Filter good matches
        good_matches = []
        for m, n in matches:
            if m.distance < 0.7 * n.distance:
                good_matches.append(m)

        # Draw the matches
        result_image = cv2.drawMatches(sample_1, keypoint1, sample, keypoint2, good_matches, None)
        print("good_matches",len(good_matches))

        # Show the result
        cv2_imshow(result_image)

551__F_Right_index_finger.BMP
/content/drive/MyDrive/SOCOFing/SOCOFing/Real/100__M_Left_index_finger.BMP
good_matches 0


NameError: ignored

In [6]:
def pixel_mse_loss(x,y):
    return tf.reduce_mean( (x - y) ** 2 )


def PSNR(y_true,y_pred):
    mse=tf.reduce_mean( (y_true - y_pred) ** 2 )
    return 20 * log10(1/ (mse ** 0.5))

def log10(x):
    numerator = tf.math.log(x)
    denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
    return numerator / denominator

def pixel_MSE(y_true,y_pred):
    return tf.reduce_mean( (y_true - y_pred) ** 2 )

In [7]:
def contrastive_loss_with_margin(margin):
  def contrastive_loss(y_true, y_pred):
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)
  return contrastive_loss

In [13]:

from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.preprocessing.image import img_to_array

# Load the SRCNN model
model = load_model('/content/drive/MyDrive/Model/srcnn_model_soco_15k.h5', custom_objects={'pixel_mse_loss': pixel_mse_loss, 'PSNR': PSNR})


best_score=0
filename=None
image =None
kp1,kp2, mp= None, None, None
for file in [file for file in os.listdir("/content/drive/MyDrive/SOCOFing/SOCOFing/Real")[:10]]:
        print(file)
        filepath ="/content/drive/MyDrive/SOCOFing/SOCOFing/Real/100__M_Left_thumb_finger.BMP"
        filepath_sample ="/content/drive/MyDrive/SOCOFing/SOCOFing/Altered/Altered-Easy/"+file.split(".")[0]+'_Zcut.BMP'
        sample_1=cv2.imread(filepath)
        sample=cv2.imread(filepath_sample)
        print(filepath)
        sift = cv2.SIFT_create()
        keypoint1, descriptor1 = sift.detectAndCompute(sample_1, None)
        keypoint2, descriptor2 = sift.detectAndCompute(sample, None)

        # FLANN parameters
        flann_params = {
            'algorithm': 1,  # FLANN_INDEX_KDTREE
            'trees': 10
        }

        # Create FLANN matcher
        flann = cv2.FlannBasedMatcher(flann_params, {})
        # Perform KNN matching
        matches = flann.knnMatch(descriptor1, descriptor2, k=2)

                # Filter good matches
        good_matches = []
        for m, n in matches:
            if m.distance < 0.7 * n.distance:
                good_matches.append(m)

        # Draw the matches
        result_image = cv2.drawMatches(sample_1, keypoint1, sample, keypoint2, good_matches, None)
        print("good_matches",len(good_matches))

        # Show the result
        cv2_imshow(result_image)

                # Load and preprocess the input image
        #input_image_path = '/content/drive/MyDrive/SOCOFing/SOCOFing/Altered/Altered-Hard/100__M_Left_index_finger_Obl.BMP'  # Replace with the path to your input image
        input_image = cv2.imread(filepath)
        input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
        input_image = cv2.resize(input_image, (256, 256))
        input_image = input_image.astype('float32') / 255.0

        # Predict the high-resolution image
        input_array = np.expand_dims(input_image, axis=0)

        predicted_image1 = model.predict(input_array)[0]



        input_image1 = cv2.imread(filepath_sample)
        input_image1 = cv2.cvtColor(input_image1, cv2.COLOR_BGR2RGB)
        input_image1 = cv2.resize(input_image1, (256, 256))
        input_image1 = input_image1.astype('float32') / 255.0

        # Predict the high-resolution image
        input_array1 = np.expand_dims(input_image1, axis=0)

        predicted_image2 = model.predict(input_array1)[0]
                # Display the input and predicted images
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.title('Predicted Image1')
        plt.imshow(predicted_image1)
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title('Predicted Image2')
        plt.imshow(predicted_image2)
        plt.axis('off')

        plt.show()
        # Save the predicted image as a file
        output_path1 = '/content/sample_data/predicted_image1.BMP'
        cv2.imwrite(output_path1, cv2.cvtColor(predicted_image1 * 255, cv2.COLOR_RGB2BGR))
        output_path1 = '/content/sample_data/predicted_image2.BMP'
        cv2.imwrite(output_path1, cv2.cvtColor(predicted_image2 * 255, cv2.COLOR_RGB2BGR))

        filepath ="/content/sample_data/predicted_image1.BMP"
        filepath_sample ="/content/sample_data/predicted_image2.BMP"
        sample_1=cv2.imread(filepath)
        sample=cv2.imread(filepath_sample)
        sift = cv2.SIFT_create()
        keypoint1, descriptor1 = sift.detectAndCompute(sample_1, None)
        keypoint2, descriptor2 = sift.detectAndCompute(sample, None)

        # FLANN parameters
        flann_params = {
            'algorithm': 1,  # FLANN_INDEX_KDTREE
            'trees': 10
        }

        # Create FLANN matcher
        flann = cv2.FlannBasedMatcher(flann_params, {})
        # Perform KNN matching
        matches = flann.knnMatch(descriptor1, descriptor2, k=2)

                # Filter good matches
        good_matches = []
        for m, n in matches:
            if m.distance < 0.7 * n.distance:
                good_matches.append(m)

        # Draw the matches
        result_image = cv2.drawMatches(sample_1, keypoint1, sample, keypoint2, good_matches, None)
        print("good_matches",len(good_matches))

        # Show the result
        cv2_imshow(result_image)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!pip3 install opencv-python